In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Hyperparameters
LR = 1e-3        # learning rate for Adam
EPOCHS = 10000    # number of training epochs (increase for better results)
N_HIDDEN = 50    # neurons per hidden layer
N_LAYERS = 4     # number of hidden layers
N_PDE = 2000     # number of interior points for PDE
N_BC = 200       # number of points per boundary segment

# Material and problem parameters
lambda_ = 1.0
mu = 0.5
Q = 4.0


In [ ]:
# -----------------------------------------------------------
# Exact (analytical) displacement and stress fields
#    for the manufactured 2D elasticity solution
# -----------------------------------------------------------

def exact_u_x(x, y):
    """
    Exact x-component of the displacement field u_x(x,y).

    Formula taken from the problem statement:
        u_x(x,y) = cos(2π x) * sin(π y)

    Inputs:
      x, y : NumPy arrays or scalars of the same shape

    Output:
      u_x  : array of same shape as x,y
    """
    return np.cos(2.0 * np.pi * x) * np.sin(np.pi * y)


def exact_u_y(x, y):
    """
    Exact y-component of the displacement field u_y(x,y):

        u_y(x,y) = sin(π x) * (Q * y^4 / 4)

    Q is a loading parameter defined globally (e.g., Q = 4).
    """
    return np.sin(np.pi * x) * (Q * (y**4) / 4.0)


def exact_sigma_xx(x, y):
    """
    Exact normal stress σ_xx(x,y) derived from exact displacements
    via the linear elasticity constitutive law:

        σ_xx = (λ + 2μ) * ε_xx + λ * ε_yy

    where
        ε_xx = ∂u_x/∂x
        ε_yy = ∂u_y/∂y

    We explicitly compute u_x_x and u_y_y using the analytical u_x, u_y.
    """
    # ε_xx = ∂u_x/∂x
    # u_x = cos(2πx) sin(πy) → ∂/∂x = -2π sin(2πx) sin(πy)
    u_x_x = -2.0 * np.pi * np.sin(2.0 * np.pi * x) * np.sin(np.pi * y)

    # ε_yy = ∂u_y/∂y
    # u_y = sin(πx) * (Q y^4 / 4) → ∂/∂y = sin(πx) * Q y^3
    u_y_y = Q * np.sin(np.pi * x) * (y**3)

    # σ_xx = (λ + 2μ) * ε_xx + λ * ε_yy
    return (lambda_ + 2 * mu) * u_x_x + lambda_ * u_y_y


def exact_sigma_yy(x, y):
    """
    Exact normal stress σ_yy(x,y):

        σ_yy = (λ + 2μ) * ε_yy + λ * ε_xx
    using the same ε_xx, ε_yy as above.
    """
    u_x_x = -2.0 * np.pi * np.sin(2.0 * np.pi * x) * np.sin(np.pi * y)
    u_y_y = Q * np.sin(np.pi * x) * (y**3)
    return (lambda_ + 2 * mu) * u_y_y + lambda_ * u_x_x


def exact_sigma_xy(x, y):
    """
    Exact shear stress σ_xy(x,y):

        σ_xy = μ * (ε_xy + ε_yx)
    with
        ε_xy = 1/2 (∂u_x/∂y + ∂u_y/∂x)
    but in this manufactured solution an equivalent closed form
    expression is already given:

        σ_xy = μ [ π cos(2πx) cos(πy)
                   + π (Q y^4 / 4) cos(πx) ]

    Inputs:
      x, y : NumPy arrays or scalars

    Output:
      σ_xy : array of same shape as x,y
    """
    return mu * (
        np.pi * np.cos(2.0 * np.pi * x) * np.cos(np.pi * y)
        + np.pi * (Q * (y**4) / 4.0) * np.cos(np.pi * x)
    )


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")


class PINN(nn.Module):
    def __init__(self, layers, neurons):
        """
        Physics-Informed Neural Network for 2D linear elasticity.

        Input  : (x, y)  ∈ [0,1]²  (spatial coordinates)
        Output : [u_x, u_y, sigma_xx, sigma_yy, sigma_xy]
                 = 5 fields predicted at each (x,y).

        layers  : number of hidden layers (depth)
        neurons : number of neurons per hidden layer (width)
        """
        super().__init__()

        # First fully connected layer: maps 2D input -> hidden features
        self.input_layer = nn.Linear(2, neurons)

        # Hidden layers: each maps (neurons -> neurons)
        # nn.ModuleList is used so that PyTorch knows these are trainable submodules.
        self.hidden = nn.ModuleList(
            [nn.Linear(neurons, neurons) for _ in range(layers - 1)]
        )

        # Output layer: maps hidden features -> 5 outputs (displacements and stresses)
        #   out[:,0] = u_x
        #   out[:,1] = u_y
        #   out[:,2] = sigma_xx
        #   out[:,3] = sigma_yy
        #   out[:,4] = sigma_xy
        self.output_layer = nn.Linear(neurons, 5)         # 5 outputs '(neuron, 5)'

        # Nonlinear activation used after each linear layer (except last).
        # Tanh is smooth and differentiable, which is useful when computing
        # many derivatives via autograd for PDE residuals.
        self.activation = nn.Tanh()
        
        # Weight initialization: Xavier normal tailored for tanh
        # This keeps variance of activations relatively stable across layers.
        for m in self.hidden:
            nn.init.xavier_normal_(m.weight, gain=nn.init.calculate_gain('tanh'))
        nn.init.xavier_normal_(self.input_layer.weight,
                               gain=nn.init.calculate_gain('tanh'))
        # Output layer typically uses gain=1 (linear output)
        nn.init.xavier_normal_(self.output_layer.weight)
        
    def forward(self, x):
        """
        Forward pass.

        x : tensor of shape (N, 2) where each row is (x_i, y_i)

        Returns:
          out : tensor of shape (N, 5)
                columns = [u_x, u_y, sigma_xx, sigma_yy, sigma_xy]
        """
        # Apply first linear layer + activation
        z = self.activation(self.input_layer(x))

        # Propagate through hidden layers, each followed by tanh
        for m in self.hidden:
            z = self.activation(m(z))

        # Final linear layer (no activation) to get 5 physical outputs
        return self.output_layer(z)


# Instantiate the model with chosen depth/width and move to CPU/GPU device
model = PINN(N_LAYERS, N_HIDDEN).to(device)



# -------------------------------------------------------------------
# Body force (source terms) f_x(x,y), f_y(x,y) used in the equilibrium
# equations: ∂σ_xx/∂x + ∂σ_xy/∂y + f_x = 0, etc.
# These are manufactured from the known exact solution.
# -------------------------------------------------------------------
def f_x_fun(x, y):
    term1 = lambda_ * (
        4.0 * np.pi**2 * torch.cos(2.0 * np.pi * x) * torch.sin(np.pi * y)
        - np.pi * torch.cos(np.pi * x) * Q * (y**3)
    )
    term2 = mu * (
        9.0 * np.pi**2 * torch.cos(2.0 * np.pi * x) * torch.sin(np.pi * y)
        - np.pi * torch.cos(np.pi * x) * Q * (y**3)
    )
    return term1 + term2


def f_y_fun(x, y):
    term1 = lambda_ * (
        -3.0 * torch.sin(np.pi * x) * Q * (y**2)
        + 2.0 * np.pi**2 * torch.sin(2.0 * np.pi * x) * torch.cos(np.pi * y)
    )
    term2 = mu * (
        -6.0 * torch.sin(np.pi * x) * Q * (y**2)
        + 2.0 * np.pi**2 * torch.sin(2.0 * np.pi * x) * torch.cos(np.pi * y)
        + (np.pi**2) * torch.sin(np.pi * x) * Q * (y**4) / 4.0
    )
    return term1 + term2



# -------------------------------------------------------------------
# Sampling of training points: interior (for PDE) and boundaries (for BC)
# -------------------------------------------------------------------

# Interior PDE collocation points: N_PDE points uniformly in [0,1]²
xy_pde = np.random.rand(N_PDE, 2)                  # shape (N_PDE, 2), each row (x,y)
xy_pde_torch = torch.tensor(xy_pde, dtype=torch.float32).to(device)

# Boundary parametric points along each side of the unit square:
# y = 0, y = 1, x = 0, x = 1

# y = 0: x ∈ [0,1]
bc_y0 = np.linspace(0, 1, N_BC)
xy_bc_y0 = np.column_stack((bc_y0, np.zeros_like(bc_y0)))  # (x, 0)

# y = 1: x ∈ [0,1]
bc_y1 = np.linspace(0, 1, N_BC)
xy_bc_y1 = np.column_stack((bc_y1, np.ones_like(bc_y1)))   # (x, 1)

# x = 0: y ∈ [0,1]
bc_x0 = np.linspace(0, 1, N_BC)
xy_bc_x0 = np.column_stack((np.zeros_like(bc_x0), bc_x0))  # (0, y)

# x = 1: y ∈ [0,1]
bc_x1 = np.linspace(0, 1, N_BC)
xy_bc_x1 = np.column_stack((np.ones_like(bc_x1), bc_x1))   # (1, y)

# Convert all boundary sets to torch tensors on the device
xy_bc_y0_torch = torch.tensor(xy_bc_y0, dtype=torch.float32).to(device)
xy_bc_y1_torch = torch.tensor(xy_bc_y1, dtype=torch.float32).to(device)
xy_bc_x0_torch = torch.tensor(xy_bc_x0, dtype=torch.float32).to(device)
xy_bc_x1_torch = torch.tensor(xy_bc_x1, dtype=torch.float32).to(device)



def loss_pde(xy):
    """
    Compute PDE residual loss over interior points.

    This PINN enforces the *system* of 5 equations:

      1) ∂σ_xx/∂x + ∂σ_xy/∂y + f_x = 0       (force balance in x)
      2) ∂σ_xy/∂x + ∂σ_yy/∂y + f_y = 0       (force balance in y)
      3) σ_xx - [ (λ+2μ)*u_xx + λ*u_yy ] = 0  (constitutive eq. for σ_xx)
      4) σ_yy - [ (λ+2μ)*u_yy + λ*u_xx ] = 0  (constitutive eq. for σ_yy)
      5) σ_xy - μ (u_x_y + u_y_x) = 0         (constitutive eq. for σ_xy)

    So stresses are not only *derived* from u, but also *output* by the network
    and constrained to satisfy these constitutive relations.
    """
    # Make a copy of xy that is detached from any previous graph,
    # and enable gradient tracking so we can differentiate w.r.t. x,y.
    xy_ = xy.clone().detach().requires_grad_(True)
    x = xy_[:, 0:1]   # (N,1) x-coordinates
    y = xy_[:, 1:2]   # (N,1) y-coordinates
    
    # Forward pass through the network
    out = model(torch.cat((x, y), dim=1))  # (N,5)
    u_x = out[:, 0:1]  # displacement component u_x(x,y)
    u_y = out[:, 1:2]  # displacement component u_y(x,y)
    sxx = out[:, 2:3]  # predicted σ_xx(x,y)
    syy = out[:, 3:4]  # predicted σ_yy(x,y)
    sxy = out[:, 4:5]  # predicted σ_xy(x,y)

    
    # -------- Equilibrium equations (1) and (2) --------
    # ∂σ_xx/∂x, ∂σ_xy/∂y, ∂σ_xy/∂x, ∂σ_yy/∂y
    sxx_x = torch.autograd.grad(sxx, x, torch.ones_like(sxx),
                                create_graph=True)[0]
    sxy_y = torch.autograd.grad(sxy, y, torch.ones_like(sxy),
                                create_graph=True)[0]
    sxy_x = torch.autograd.grad(sxy, x, torch.ones_like(sxy),
                                create_graph=True)[0]
    syy_y = torch.autograd.grad(syy, y, torch.ones_like(syy),
                                create_graph=True)[0]

    # Body forces at (x,y)
    fx = f_x_fun(x, y)
    fy = f_y_fun(x, y)

    ## Residuals of momentum balance
    # Resiudal 1 = ∂σ_xx/∂x + ∂σ_xy/∂y + f_x
    eq1 = sxx_x + sxy_y + fx  # should be ≈ 0
    # Resiudal 2 = ∂σ_yy/∂y + ∂σ_yx/∂x + f_y
    eq2 = sxy_x + syy_y + fy  # should be ≈ 0 

    
    # -------- Constitutive equations (3), (4), (5) --------
    # First derivatives of displacements
    u_x_x = torch.autograd.grad(u_x, x, torch.ones_like(u_x),
                                create_graph=True)[0]
    u_y_y = torch.autograd.grad(u_y, y, torch.ones_like(u_y),
                                create_graph=True)[0]
    u_x_y = torch.autograd.grad(u_x, y, torch.ones_like(u_x),
                                create_graph=True)[0]
    u_y_x = torch.autograd.grad(u_y, x, torch.ones_like(u_y),
                                create_graph=True)[0]

    ## Constitutive residuals:
    #   Residual 3) σ_xx - ((λ+2μ) u_xx + λ u_yy) = 0  ~ Residual 3
    eq3 = sxx - ((lambda_ + 2 * mu) * u_x_x + lambda_ * u_y_y)
    
    #   Residual 4) σ_yy - ((λ+2μ) u_yy + λ u_xx) = 0  ~ Residual 4
    eq4 = syy - ((lambda_ + 2 * mu) * u_y_y + lambda_ * u_x_x)

    #   Residual 5) σ_xy - μ (u_xy + u_yx) = 0         ~ Residual 5    
    eq5 = sxy - mu * (u_x_y + u_y_x)

    # Total PDE loss = mean squared residual of all 5 equations
    return torch.mean(eq1**2 + eq2**2 + eq3**2 + eq4**2 + eq5**2)



def loss_bc():
    """
    Boundary-condition loss.

    We enforce displacement and traction conditions by using the
    network outputs {u_x, u_y, σ_xx, σ_yy, σ_xy} at boundary points.

    Conditions:

      y = 0 : u_x = 0, u_y = 0
      y = 1 : u_x = 0, σ_yy = (λ+2μ) Q sin(π x)
      x = 0 : u_y = 0, σ_xx = 0
      x = 1 : u_y = 0, σ_xx = 0
    """
    loss_val = 0.0
    
    # ---------------- y = 0 : u_x = 0, u_y = 0 ----------------
    x0 = xy_bc_y0_torch[:, 0:1].clone().detach().requires_grad_(True)
    y0 = xy_bc_y0_torch[:, 1:2].clone().detach().requires_grad_(True)
    out_y0 = model(torch.cat((x0, y0), dim=1))  # (N_BC,5)
    # Penalize non-zero displacement
    loss_val += torch.mean(out_y0[:, 0:1]**2)  # u_x -> 0
    loss_val += torch.mean(out_y0[:, 1:2]**2)  # u_y -> 0
    
    # ---------------- y = 1 : u_x = 0, σ_yy = (λ+2μ) Q sin(π x) ----------------
    x1 = xy_bc_y1_torch[:, 0:1].clone().detach().requires_grad_(True)
    y1 = xy_bc_y1_torch[:, 1:2].clone().detach().requires_grad_(True)
    out_y1 = model(torch.cat((x1, y1), dim=1))
    # u_x(x,1) -> 0
    loss_val += torch.mean(out_y1[:, 0:1]**2)
    # σ_yy at y=1
    syy_1 = out_y1[:, 3:4]
    syy_target = (lambda_ + 2 * mu) * Q * torch.sin(np.pi * x1)
    loss_val += torch.mean((syy_1 - syy_target)**2)
    
    # ---------------- x = 0 : u_y = 0, σ_xx = 0 ----------------
    x0_ = xy_bc_x0_torch[:, 0:1].clone().detach().requires_grad_(True)
    y0_ = xy_bc_x0_torch[:, 1:2].clone().detach().requires_grad_(True)
    out_x0 = model(torch.cat((x0_, y0_), dim=1))
    # u_y(0,y) -> 0
    loss_val += torch.mean(out_x0[:, 1:2]**2)
    # σ_xx(0,y) -> 0  (component index 2)
    loss_val += torch.mean(out_x0[:, 2:3]**2)
    
    # ---------------- x = 1 : u_y = 0, σ_xx = 0 ----------------
    x1_ = xy_bc_x1_torch[:, 0:1].clone().detach().requires_grad_(True)
    y1_ = xy_bc_x1_torch[:, 1:2].clone().detach().requires_grad_(True)
    out_x1 = model(torch.cat((x1_, y1_), dim=1))
    # u_y(1,y) -> 0
    loss_val += torch.mean(out_x1[:, 1:2]**2)
    # σ_xx(1,y) -> 0
    loss_val += torch.mean(out_x1[:, 2:3]**2)

    return loss_val



# -------------------------------------------------------------------
# Training loop: minimize PDE residual + BC residual
# -------------------------------------------------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for ep in range(EPOCHS):
    optimizer.zero_grad()

    # Interior (PDE) residual loss
    loss_interior = loss_pde(xy_pde_torch)

    # Boundary condition loss on all four sides
    loss_bound = loss_bc()

    # Total loss (here simple sum, weights could be tuned)
    loss_total = loss_interior + loss_bound

    # Backpropagation
    loss_total.backward()
    optimizer.step()

    if ep % 500 == 0:
        print(
            f"Epoch {ep:5d} | "
            f"PDE Loss: {loss_interior.item():.4e} | "
            f"BC Loss: {loss_bound.item():.4e} | "
            f"Total: {loss_total.item():.4e}"
        )

print("Training finished.")





optimizer = torch.optim.Adam(model.parameters(), lr=LR)

for ep in range(EPOCHS):
    optimizer.zero_grad()
    loss_interior = loss_pde(xy_pde_torch)
    loss_bound = loss_bc()
    loss_total = loss_interior + loss_bound
    loss_total.backward()
    optimizer.step()

    if ep % 500 == 0:
        print(f"Epoch {ep:5d} | PDE Loss: {loss_interior.item():.4e} | BC Loss: {loss_bound.item():.4e} | Total: {loss_total.item():.4e}")

print("Training finished.")







In [ ]:
# -----------------------------------------------------------
# Build evaluation grid and run the trained PINN
# -----------------------------------------------------------

# Stack the grid coordinates XX, YY into a list of (x, y) pairs.
#   XX, YY have shape (ny, nx).
#   XX.ravel(), YY.ravel() → 1D arrays of length ny*nx.
#   np.column_stack → array of shape (ny*nx, 2): [(x_1, y_1), ..., (x_N, y_N)].
XY_plot = np.column_stack((XX.ravel(), YY.ravel()))

# Convert the evaluation points into a torch tensor on the correct device.
XY_torch = torch.tensor(XY_plot, dtype=torch.float32).to(device)

# Disable gradient tracking during inference to save memory and computation.
with torch.no_grad():
    # Forward pass through the PINN at all evaluation points.
    # Output shape: (ny*nx, 5) = [u_x, u_y, sxx, syy, sxy].
    pred = model(XY_torch).cpu().numpy()  # move to CPU and convert to NumPy

# Split the 5 outputs and reshape back to (ny, nx) grids for contour plots.
u_x_pred = pred[:, 0].reshape(ny, nx)
u_y_pred = pred[:, 1].reshape(ny, nx)
sxx_pred = pred[:, 2].reshape(ny, nx)
syy_pred = pred[:, 3].reshape(ny, nx)
sxy_pred = pred[:, 4].reshape(ny, nx)

# Exact (analytical) solutions on the same grid (XX, YY).
# These functions should return arrays with shape (ny, nx).
u_x_ex = exact_u_x(XX, YY)
u_y_ex = exact_u_y(XX, YY)
sigma_xx_exact = exact_sigma_xx(XX, YY)
sigma_yy_exact = exact_sigma_yy(XX, YY)
sigma_xy_exact = exact_sigma_xy(XX, YY)


# -----------------------------------------------------------
# Compare u_x : PINN vs exact solution
# -----------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Left: PINN-predicted displacement u_x(x,y)
cont1 = axs[0].contourf(XX, YY, u_x_pred, levels=30)
fig.colorbar(cont1, ax=axs[0])
axs[0].set_title("PINN predicted $u_x$")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

# Right: exact displacement u_x(x,y)
cont2 = axs[1].contourf(XX, YY, u_x_ex, levels=30)
fig.colorbar(cont2, ax=axs[1])
axs[1].set_title("Exact $u_x$")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")

plt.tight_layout()
plt.show()


# -----------------------------------------------------------
# Compare u_y : PINN vs exact solution
# -----------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Left: PINN-predicted displacement u_y(x,y)
cont3 = axs[0].contourf(XX, YY, u_y_pred, levels=30)
fig.colorbar(cont3, ax=axs[0])
axs[0].set_title("PINN predicted $u_y$")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

# Right: exact displacement u_y(x,y)
cont4 = axs[1].contourf(XX, YY, u_y_ex, levels=30)
fig.colorbar(cont4, ax=axs[1])
axs[1].set_title("Exact $u_y$")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")

plt.tight_layout()
plt.show()


# -----------------------------------------------------------
# Compare σ_xx : PINN vs exact solution
# -----------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Left: PINN-predicted normal stress σ_xx(x,y)
cont5 = axs[0].contourf(XX, YY, sxx_pred, levels=30)
fig.colorbar(cont5, ax=axs[0])
axs[0].set_title("PINN predicted $\sigma_{xx}$")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

# Right: exact σ_xx(x,y)
cont6 = axs[1].contourf(XX, YY, sigma_xx_exact, levels=30)
fig.colorbar(cont6, ax=axs[1])
axs[1].set_title("Exact $\sigma_{xx}$")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")

plt.tight_layout()
plt.show()


# -----------------------------------------------------------
# Compare σ_yy : PINN vs exact solution
# -----------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Left: PINN-predicted normal stress σ_yy(x,y)
cont7 = axs[0].contourf(XX, YY, syy_pred, levels=30)
fig.colorbar(cont7, ax=axs[0])
axs[0].set_title("PINN predicted $\sigma_{yy}$")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

# Right: exact σ_yy(x,y)
cont8 = axs[1].contourf(XX, YY, sigma_yy_exact, levels=30)
fig.colorbar(cont8, ax=axs[1])
axs[1].set_title("Exact $\sigma_{yy}$")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")

plt.tight_layout()
plt.show()


# -----------------------------------------------------------
# Compare σ_xy : PINN vs exact solution
# -----------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Left: PINN-predicted shear stress σ_xy(x,y)
cont9 = axs[0].contourf(XX, YY, sxy_pred, levels=30)
fig.colorbar(cont9, ax=axs[0])
axs[0].set_title("PINN predicted $\sigma_{xy}$")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

# Right: exact σ_xy(x,y)
cont10 = axs[1].contourf(XX, YY, sigma_xy_exact, levels=30)
fig.colorbar(cont10, ax=axs[1])
axs[1].set_title("Exact $\sigma_{xy}$")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")

plt.tight_layout()
plt.show()
